### Can we classify each phase as the stress vs non-stress phase for each subject? ###

In [3]:
# IMPORTING MODULES
import glob
import importlib
import matplotlib.pyplot as plt
import numpy as np
import os
cvx_path = os.path.abspath(os.path.join('..', '..', 'cvxEDA', 'src'))
module_path = os.path.abspath(os.path.join('..', '..', 'src'))
sys.path.append(module_path)
import pandas as pd
import random
import scipy.signal as ss
import sys

import tools.data_reader_sfi as dr
import tools.display_tools as dt
import tools.preprocessing as preprocessing
import train

from scipy.fft import fft, fftfreq, fftshift
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import train_test_split, cross_val_score, RepeatedKFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import normalize

import cvxopt.solvers
cvxopt.solvers.options['show_progress'] = False

import warnings
warnings.filterwarnings(
    "ignore", 
    category=RuntimeWarning
)

stress_phases = [f"CLIP-{str(phase)}" for phase in dr.CLIP_INDICES]
rest_phase = ["BIOFEEDBACK-REST"]


In [31]:
importlib.reload(train)
importlib.reload(dt)
importlib.reload(preprocessing)


metrics = [
    train.Metrics.BPM, 
    train.Metrics.RMSSD, 
    train.Metrics.HF_RR, 
    train.Metrics.LF_RR, 
    train.Metrics.IBI, 
    train.Metrics.SDNN, 
    train.Metrics.MEAN_SCL, 
    train.Metrics.SCR_RATE, 
    train.Metrics.RESP,
    # train.Metrics.MEAN_WRIST_ACT, 
    # train.Metrics.PEAK_WRIST_ACC 
]

model_phases = stress_phases + rest_phase

normalize = True
# normalize = False
x, y = train.Train_SFI.get_sfi_data(metrics, model_phases, verbose=False, normalize=normalize)
# 0-1 scaling
for i in range(2, len(x.columns)):
    if x.columns[i] in metrics:
        data_col = x[x.columns[i]]
        data_col = (data_col - data_col.min())/(data_col.max() - data_col.min())
        x[x.columns[i]] = data_col

models = {
    "SVM": SVC(C=10, gamma=0.01), 
    "KNN": KNeighborsClassifier(n_neighbors=5),
    "LogReg": LogisticRegression(max_iter=1000),
    "Bayes": GaussianNB(),
    "DT": DecisionTreeClassifier()
}

results = {
    "SVM": [], 
    "KNN": [],
    "LogReg": [],
    "Bayes": [],
    "DT": []
}
num_iters = 10
for _ in range(num_iters):
    out = train.train_predict(models, x, y, by_subject=False, test_size=0.12, show_classification=False)
    for model_name in results:
        results[model_name].append(out[model_name])

for model_name in results.keys():
    print(f"{model_name} accuracy over {num_iters} rounds: {np.mean(results[model_name])}")

    subject      CLIP-1      CLIP-2      CLIP-3      CLIP-4      CLIP-5  \
0         2   93.820933   88.678193   88.456189   84.603139   91.254753   
1         3   74.735605   71.580741   72.865275   70.897656   75.997467   
2         5   78.358209   81.351690   78.226858   79.714456   78.455791   
3         6   64.907952   70.465420   72.566711   67.664787   73.455111   
4         8   92.232015   94.348894   90.255950   91.566265   94.112591   
5         9   92.271293   90.312621   87.184662   94.497852   86.477321   
6        11   70.306657   71.341907   68.101206   68.994889  107.051827   
7        12   78.774617   77.701907  180.000000   73.035780   73.346693   
8        14   82.936130   83.696600   79.768786   76.606795   78.338279   
9        15   78.096400   78.998489   83.966413  255.319149   74.788902   
10       17   73.390816   75.503356   75.471698   76.255031   78.001219   
11       18   75.642245   79.098204   74.454429   76.766856   73.793756   
12       20  105.218677  

ValueError: Length of values (8) does not match length of index (56)